In [ ]:
import cv2
import numpy as np

# def linear_stretching(input, lower_stretch_from, upper_stretch_from):
#     lower_stretch_to = 0
#     upper_stretch_to = 255
#     output = (input - lower_stretch_from) * ((upper_stretch_to - lower_stretch_to) / (upper_stretch_from - lower_stretch_from)) + lower_stretch_to
#     return output

# def gamma_correction():
#     print("On-Run: Gamma_correction")
#     moon = cv2.imread("./image/Show1_Gamma_Input.jpg", 0)
    
#     max_value = np.max(moon)
#     min_value = np.min(moon)
    
#     print("On-Run: Linear_stretching")
#     for y in range(len(moon)):
#         for x in range(len(moon[y])):
#             moon[y][x] = linear_stretching(moon[y][x], min_value, max_value)
            
#     cv2.imwrite("./image/Show2_Gamma_Output.jpg", moon)
#     print("Finish: Linear_stretching")
#     print("Finish: Gamma_correction")
    
def detect_coins():
    coins = cv2.imread("./image/Show1_Input.jpg", 1)
    gray = cv2.cvtColor(coins, cv2.COLOR_BGR2GRAY)
    img = cv2.medianBlur(gray, 7)
    circles = cv2.HoughCircles(
        img, cv2.HOUGH_GRADIENT,
        1, 50, param1=100, param2=50,
        minRadius=10, maxRadius=80
    )
    
    coins_copy = coins.copy()
    for detected_circle in circles[0]:
        x, y, detected_radius = detected_circle
        coins_detected = cv2.circle(
            coins_copy, (int(x), int(y)), 
            int(detected_radius), (0,255,0), 4
        )
    
    cv2.imwrite("./image/Show2_Output_Hough.jpg", coins_detected)
    return circles

def calculate_amount():
    koruny = {
        "1 CZK": {
            "value": 1,
            "radius": 20,
            "ratio": 1,
            "count": 0,
        },
        "2 CZK": {
            "value": 2,
            "radius": 21.5,
            "ratio": 1.075,
            "count": 0,
        },
        "5 CZK": {
            "value": 5,
            "radius": 23,
            "ratio": 1.15,
            "count": 0,
        },
        "10 CZK": {
            "value": 10,
            "radius": 24.5,
            "ratio": 1.225,
            "count": 0,
        },
        "20 CZK": {
            "value": 20,
            "radius": 26,
            "ratio": 1.3,
            "count": 0,
        },
        "50 CZK": {
            "value": 50,
            "radius": 27.5,
            "ratio": 1.375,
            "count": 0,
        },
    }

    print("On-Run: Calulate_amount")
    circles = detect_coins()
    radius = []
    coordinates = []
    
    for detected_circle in circles[0]:
        x, y, detected_radius = detected_circle
        radius.append(detected_radius)
        coordinates.append([x, y])
        
    smallest = min(radius)
    tolerance = 0.0375
    total_amount = 0
    
    coins_circled = cv2.imread("./image/Show2_Output_Hough.jpg", 1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    for coin in circles[0]:
        ratio_to_check = coin[2] / smallest
        coor_x = coin[0]
        coor_y = coin[1]
        for koruna in koruny:
            value = koruny[koruna]["value"]
            if(abs(ratio_to_check - koruny[koruna]["ratio"]) <= tolerance):
                koruny[koruna]["count"] += 1
                total_amount += koruny[koruna]["value"]
                cv2.putText(
                    coins_circled, str(value), (int(coor_x), int(coor_y)),
                    font, 1, (0,0,0), 4
                )
                
    print(f"The total amount is {total_amount} CZK")
    for koruna in koruny:
        pieces = koruny[koruna]["count"]
        print(f"{koruna}={pieces}x")
        
    cv2.imwrite("./image/Show3_Output_Count.jpg", coins_circled)
    print("Finish: Calculate_amount")
    
if __name__ == "__main__":
#     picTest = cv2.imread("./image/koruny_r11.jpg") # work
#     picTest = cv2.imread("./image/koruny_t10.jpg") # work
#     picTest = cv2.imread("./image/koruny_t20.jpg") # not work
    picTest = cv2.imread("./image/koruny_2.jpg") # not work
    
    cv2.imwrite("./image/Show1_Input.jpg", picTest)
#     gamma_correction()
    calculate_amount()
    coins1 = cv2.imread("./image/Show1_Input.jpg")
    coins2 = cv2.imread("./image/Show2_Output_Hough.jpg")
    coins3 = cv2.imread("./image/Show3_Output_Count.jpg")
    
    cv2.imshow("Show1_Coin_Input", coins1)
    cv2.imshow("Show2_Hough_Output", coins2)
    cv2.imshow("Show3_Count_Output", coins3)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
        

On-Run: Calulate_amount
The total amount is 3 CZK
1 CZK=1x
2 CZK=1x
5 CZK=0x
10 CZK=0x
20 CZK=0x
50 CZK=0x
Finish: Calculate_amount
